In [7]:
import pygfx as gfx
import numpy as np
import wgpu
from wgpu.gui.auto import WgpuCanvas, run
from IPython.display import display

class CustomShader(gfx.Material):
    def __init__(self):
        super().__init__()
        self.spiral_color = (0, 1, 0)  # Green
        self.bg_color = (0, 0, 0)  # Black
        self.spin_speed = 1.0
        self.throb_speed = 1.0
        self.throb_strength = 1.0
        self.zoom = 1.0
        self.time = 0.0
        self.i_res = (1024, 1024)

    def get_render_function(self, wobject, renderer):
        code = self.get_code()
        bindings = self.get_bindings(wobject, renderer)
        
        render_info = {
            "vertex_shader": code,
            "fragment_shader": code,
            "bindings": bindings,
        }
        
        return gfx.renderers.wgpu.render_functions.render_mesh, render_info

    def get_code(self):
        return """
        struct Uniforms {
            u_spiral_color: vec3<f32>,
            u_bg_color: vec3<f32>,
            u_spin_speed: f32,
            u_throb_speed: f32,
            u_throb_strength: f32,
            u_zoom: f32,
            u_time: f32,
            u_i_res: vec2<f32>,
        };
        @group(0) @binding(0) var<uniform> uniforms: Uniforms;

        struct VertexInput {
            @location(0) position: vec3<f32>,
        };

        struct VertexOutput {
            @builtin(position) position: vec4<f32>,
            @location(0) uv: vec2<f32>,
        };

        @vertex
        fn vs_main(input: VertexInput) -> VertexOutput {
            var output: VertexOutput;
            output.position = vec4<f32>(input.position, 1.0);
            output.uv = input.position.xy * 0.5 + 0.5;
            return output;
        }

        @fragment
        fn fs_main(input: VertexOutput) -> @location(0) vec4<f32> {
            let iTime = uniforms.u_time * 0.001;
            let truPos = vec2<f32>(1.0, uniforms.u_i_res.y / uniforms.u_i_res.x) * (input.uv - vec2<f32>(0.5));

            let angle = atan2(truPos.y, truPos.x);
            let dist = pow(length(truPos), 0.4 + sin((iTime + cos(iTime * 0.05) * 0.1) * uniforms.u_throb_speed) * 0.2);

            var spiFactor = pow(sin(angle + dist * 40.0 * uniforms.u_zoom - iTime * 5.0 * uniforms.u_spin_speed) + 1.0, 50.0);
            spiFactor = clamp(spiFactor, 0.0, 1.0);

            let color = mix(uniforms.u_spiral_color, uniforms.u_bg_color, spiFactor);
            return vec4<f32>(color, 1.0);
        }
        """

    def get_bindings(self, wobject, renderer):
        return {
            "uniforms": {
                "type": "uniform",
                "struct": {
                    "u_spiral_color": self.spiral_color,
                    "u_bg_color": self.bg_color,
                    "u_spin_speed": self.spin_speed,
                    "u_throb_speed": self.throb_speed,
                    "u_throb_strength": self.throb_strength,
                    "u_zoom": self.zoom,
                    "u_time": self.time,
                    "u_i_res": self.i_res,
                },
            }
        }

# Create a custom shader material
custom_shader = CustomShader()

# Create a geometry for a full-screen quad
geometry = gfx.Geometry(
    positions=np.array([[-1, -1, 0], [1, -1, 0], [-1, 1, 0], [1, 1, 0]], dtype=np.float32),
    indices=np.array([0, 1, 2, 1, 3, 2], dtype=np.uint32),
)

# Create a mesh using the geometry and custom shader
quad = gfx.Mesh(geometry, custom_shader)

# Create a scene and add the quad
scene = gfx.Scene()
scene.add(quad)

# Create a camera (orthographic for full-screen quad)
camera = gfx.OrthographicCamera(2, 2)

# Create a canvas
canvas = WgpuCanvas(size=(800, 600))

# Create a renderer
renderer = gfx.WgpuRenderer(canvas)

# Display the canvas
display(canvas)

# Render loop
def animate(time):
    custom_shader.time = time
    renderer.render(scene, camera)
    canvas.request_draw()

# Start the animation
canvas.request_animation(animate)

RFBOutputContext()

Present error: present() is called without a preceeding call to get_c… (5)


JupyterWgpuCanvas(css_height='600px', css_width='800px')

AttributeError: 'JupyterWgpuCanvas' object has no attribute 'request_animation'

Present error: present() is called without a preceeding call to get_c… (6)
Present error: present() is called without a preceeding call to get_c… (7)
Present error: present() is called without a preceeding call to get_c… (8)
Present error: present() is called without a preceeding call to get_c… (9)
Present error: present() is called without a preceeding call to get_c… (10)
Present error: present() is called without a preceeding call to get_c… (11)
Present error: present() is called without a preceeding call to get_c… (12)
Present error: present() is called without a preceeding call to get_c… (13)
Present error: present() is called without a preceeding call to get_c… (14)
Present error: present() is called without a preceeding call to get_c… (15)
Present error: present() is called without a preceeding call to get_c… (16)


## For debugging
https://blog.jupyter.org/ipycanvas-a-python-canvas-for-jupyter-bbb51e4777f7

https://docs.pygfx.org/stable/_autosummary/renderers/wgpu/pygfx.renderers.wgpu.WgpuRenderer.html

https://docs.pygfx.org/stable/_gallery/index.html#introductory-examples

In [6]:
print(canvas)

JupyterWgpuCanvas(css_height='600px', css_width='800px')
